# STEP #2
Baseline Neural Network Model Performance

In [1]:
import numpy
import pandas
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
seed = 7
numpy.random.seed(seed)

In [3]:
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
x = dataset[:,0:60].astype(float)
Y = dataset[:,60]
encoder = LabelEncoder()
A = encoder.fit_transform(Y)

In [4]:
def create_baseline():
    model = tensorflow.keras.Sequential()
    model.add(layers.Dense(60 , activation='relu' , input_shape=(60 ,)))
    model.add(layers.Dense(1 , activation='sigmoid'))
    
    model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])

    return model

In [5]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, A, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 82.23% (6.72%)


# STEP 3
Re-Run The Baseline Model With Data Preparation

In [6]:
def create_baseline():
    model = tensorflow.keras.Sequential()
    model.add(layers.Dense(60 , activation='relu' , input_shape=(60 ,)))
    model.add(layers.Dense(1 , activation='sigmoid'))
    
    model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])

    return model

In [7]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, A, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 84.09% (5.74%)


# STEP #4
Tuning Layers and Number of Neurons in The Model

# STEP 4.1
Evaluate a Smaller Network

In [8]:
def create_smaller():
    
    model = tensorflow.keras.Sequential()
    model.add(layers.Dense(30 , activation='relu' , input_shape=(60 ,)))
    model.add(layers.Dense(1 , activation='sigmoid'))
    
    model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])
    
    return model

In [9]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, A, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))




Standardized: 85.09% (5.01%)


# 4.2
Evaluate a Larger Network

In [10]:
def create_larger():
    
    model = tensorflow.keras.Sequential()
    model.add(layers.Dense(60 , activation='relu' , input_shape=(60 ,)))
    model.add(layers.Dense(30 , activation='relu' ))
    model.add(layers.Dense(1 , activation='sigmoid'))
    
    model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])
    
    return model

In [11]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, A, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))




Standardized: 84.09% (6.12%)


# STEP #5
Really Scaling up: developing a model that overfits

In [12]:
def create_baseline():
    model = tensorflow.keras.Sequential()
    model.add(layers.Dense(60 , activation='relu' , input_shape=(60 ,)))
    model.add(layers.Dense(30 , activation='relu' ))
    model.add(layers.Dense(30 , activation='relu' ))
    model.add(layers.Dense(16 , activation='relu' ))
    model.add(layers.Dense(1 , activation='sigmoid'))
    
    model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])
    
    return model

In [13]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, A, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))




Standardized: 85.04% (5.62%)


# STEP #6
Tuning the Model

In [14]:
def create_baseline():
    model = tensorflow.keras.Sequential()
    model.add(layers.Dense( 120, activation='relu' , input_shape=(60 ,)))
    model.add(layers.Dense(1 , activation='sigmoid'))
    
    model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])
    
    return model

In [15]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, A, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))




Standardized: 83.64% (5.75%)


# STEP #7
Rewriting the code using the Keras Functional API

In [4]:
inputs= tensorflow.keras.Input(shape=(60,))
X = layers.Dense(60 , activation='relu')(inputs)
outputs = layers.Dense(1 , activation='sigmoid')(X)
    
model = tensorflow.keras.Model(inputs , outputs)

In [5]:
model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=[('accuracy')])
model.fit(x , A , epochs =100 , batch_size =5)

Train on 208 samples
Epoch 1/100
208/208 [==============================] - 2s 11ms/sample - loss: 0.7099 - accuracy: 0.4904
Epoch 2/100
208/208 [==============================] - 0s 923us/sample - loss: 0.6687 - accuracy: 0.5913
Epoch 3/100
208/208 [==============================] - 0s 917us/sample - loss: 0.6407 - accuracy: 0.6731
Epoch 4/100
208/208 [==============================] - 0s 817us/sample - loss: 0.6135 - accuracy: 0.7115
Epoch 5/100
208/208 [==============================] - 0s 846us/sample - loss: 0.5961 - accuracy: 0.7163
Epoch 6/100
208/208 [==============================] - 0s 711us/sample - loss: 0.5718 - accuracy: 0.7212
Epoch 7/100
208/208 [==============================] - 0s 676us/sample - loss: 0.5501 - accuracy: 0.7644
Epoch 8/100
208/208 [==============================] - 0s 726us/sample - loss: 0.5312 - accuracy: 0.7644
Epoch 9/100
208/208 [==============================] - 0s 692us/sample - loss: 0.5108 - accuracy: 0.7837
Epoch 10/100
208/208 [=============

# STEP #8
Rewriting the code by doing Model Subclassing

In [7]:
class MyModel(tensorflow.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1=layers.Dense(30 , activation='relu')
        self.dense2=layers.Dense(1 , activation='sigmoid')
        
    def call(self , inputs):
        x = self.dense1(inputs)
        return self.dense2(x)
    
model=MyModel() 

In [8]:
model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])
model.fit(X , A, epochs = 100, batch_size = 5)

Train on 208 samples
Epoch 1/100
208/208 [==============================] - 1s 6ms/sample - loss: 0.6705 - accuracy: 0.5721
Epoch 2/100
208/208 [==============================] - 0s 1ms/sample - loss: 0.6444 - accuracy: 0.6058
Epoch 3/100
208/208 [==============================] - 0s 905us/sample - loss: 0.6171 - accuracy: 0.7308
Epoch 4/100
208/208 [==============================] - 0s 664us/sample - loss: 0.5997 - accuracy: 0.7308
Epoch 5/100
208/208 [==============================] - 0s 646us/sample - loss: 0.5768 - accuracy: 0.7596
Epoch 6/100
208/208 [==============================] - 0s 376us/sample - loss: 0.5613 - accuracy: 0.7644
Epoch 7/100
208/208 [==============================] - 0s 388us/sample - loss: 0.5428 - accuracy: 0.7885
Epoch 8/100
208/208 [==============================] - 0s 404us/sample - loss: 0.5253 - accuracy: 0.7933
Epoch 9/100
208/208 [==============================] - 0s 405us/sample - loss: 0.5137 - accuracy: 0.7933
Epoch 10/100
208/208 [================